In [ ]:
!pip install pyngrok
!pip install DeepFace
!pip install flask-bcrypt
!pip install flask-mysqldb
from deepface import DeepFace
import matplotlib.pyplot as plt
import io
from PIL import Image
from pyngrok import ngrok
from flask import Flask,Response,request,render_template,redirect,url_for,session
from flask_bcrypt import Bcrypt
from flask_mysqldb import MySQL
from google.colab import drive,userdata
import time,datetime,requests,subprocess,base64,hashlib,os,pandas as pd

drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def getID(value, mod):
    hashObj = hashlib.sha256()
    hashObj.update(value.encode('utf-8'))
    hexHash= hashObj.hexdigest()
    ID = int(hexHash,16) % mod
    return ID



def DBconnect(app):
    app.secret_key = 'kjdjkshljKLKJKLWWjsqqqPOllmJA$%&*^&'
    app.config['MYSQL_HOST'] = '86.5.64.82'
    app.config['MYSQL_USER'] = 'faceRecUser'
    app.config['MYSQL_PASSWORD'] = 'ALTV_face_rec_admin3306'
    app.config['MYSQL_PORT'] = 3306
    app.config['MYSQL_DB'] = 'faceRecognition'
    mysql = MySQL(app)
    return mysql

def getCurrentEvent(db, studentID):
    cursor = db.connection.cursor()
    sql = "SELECT e.eventID FROM event e JOIN attendance a ON e.eventID = a.eventID WHERE a.studentID = %s AND e.eventStart < NOW() AND NOW() < DATE_ADD(eventStart,INTERVAL 1 HOUR);"
    cursor.execute(sql)
    return cursor.fetchone()


def MakeEvent(db,location,startTime,teacherName):
    teacherID = getID(teacherName,10000000)
    cursor = db.connection.cursor()
    cursor.execute("INSERT INTO event(eventID, location,eventStart,teacher) VALUES(%s,%s,%s,%s)",[getID(location+teacherName+startTime,10000000),location,startTime,teacherID])
    db.connection.commit()
    print(cursor.rowcount, "record inserted.")

def addSeminarGroupToEvent(db,eventID,seminarGroup):
    cursor = db.connection.cursor()
    cursor.execute("SELECT studentID FROM student WHERE seminarGroup = %s ",(seminarGroup,))
    students = cursor.fetchall()
    for student in students:
        sql = "INSERT INTO attendance(studentID,eventID,attended,timeArrived) VALUES(%s,%s,%s,%s)"
        cursor.execute(sql,(student[0],eventID,False,None))
        db.connection.commit()
    print(cursor.rowcount, "record inserted.")

def AddTeacher(db,teacherName, password):
    print(teacherName)
    cursor = db.connection.cursor()
    sql = "INSERT INTO teacher(teacherID,password)  VALUES(%s,%s)"
    try:
        cursor.execute(sql, (getID(teacherName,10000000), bcrypt.generate_password_hash(password=password)))
        db.connection.commit()
        print(cursor.rowcount, "record inserted.")
        return True
    except Exception as e:
        print("cannot record data error s%", e)
        return False
    
def validateTeacher(db,bcrypt,teacherName, password):
    cursor = db.connection.cursor()
    try:
        cursor.execute("SELECT * FROM teacher WHERE teacherID = %s;",(getID(teacherName,10000000),))
        account = cursor.fetchone()
        print(account[0])
        print(account[1])
        if account == None:
            print("No account found")
            return None
        if bcrypt.check_password_hash(account[1],password):
            return True
        else:
            return False
    except Exception as e:
        print('error: %s',e)


def updateAttendence(db,student,eventID):
    sql = "UPDATE attendance SET attended = %s,timeArrived = NOW() WHERE studentID = %s AND eventID = %s"
    values = (True,str(student),eventID,)
    cursor = db.connection.cursor()
    cursor.execute(sql,values)
    print(values)
    print(cursor.rowcount, "record ammended.")
    db.connection.commit()

def getStudentAttendanceStats(db,student):
    db = db.connection
    print(student)
    sql = "SELECT * FROM attendance WHERE studentID = " + str(student)
    df = pd.read_sql_query(sql = sql, con=db)
    print(df)
    return df


def addStudent(db,studentName,seminarGroup):
    cursor = db.connection.cursor()
    sql = "INSERT INTO student(studentID,studentName,seminarGroup) VALUES(%s,%s,%s)"
    if studentName == None:
        print("no student name added")
        return None
    else:
        studentID = getID(studentName,10000000)
        try:
            cursor.execute(sql,(studentID,studentName,seminarGroup))
            db.connection.commit()
            print(cursor.rowcount, "record inserted.")
            return True
        except Exception as e:
            print("cannot record data error s%", e)
            return False
        
def addSeminarGroup(db,seminarGroup):
    cursor = db.connection.cursor()
    sql = "INSERT INTO seminarGroup(groupID) VALUES(%s)"
    if seminarGroup == None:
        print("no student name added")
        return None
    else:
        try:
            cursor.execute(sql,(seminarGroup,))
            db.connection.commit()
            print(cursor.rowcount, "record inserted.")
            return True
        except Exception as e:
            print("cannot record data error s%", e)
            return False

def getAttendancePie(mysql,studentName):
    print("######### MAKING PIE CHART########")
    df = getStudentAttendanceStats(mysql,getID(studentName,10000000))
    print("######### FINISHED GETTING DATA########")
    attendance_count = df['attended'].value_counts()

    # Labels for the pie chart
    labels = ['Not Attended','Attended']

    # Create pie chart
    plt.figure(figsize=(7, 7))
    plt.pie(attendance_count, labels=labels, autopct='%1.1f%%', startangle=90, colors=['#66b3ff', '#ff6666'])
    plt.title('Student Attendance Distribution')
    plt.axis('equal')
    if not os.path.exists("images/attendance_pie_chart.png"):
        os.mkdir("images")
    os.chdir("images")
    plt.savefig('attendance_pie_chart.png', bbox_inches='tight')

def getStudentName(db,studentID):
    cur = db.connection.cursor()
    cur.execute(f"SELECT studentName from student WHERE studentID = {studentID};")
    name = cur.fetchall()[0][0]
    cur.close()
    return name.capitalize()


In [ ]:
filepath = userdata.get("attendance_filepath")
ngrok.set_auth_token(userdata.get("ngrok_token"))
app = Flask(__name__,template_folder= filepath+"templates",static_folder=filepath+"static")
bcrypt = Bcrypt(app)
mysql = DBconnect(app)

def start_ngrok():
    ngrok_process = subprocess.Popen(['ngrok', 'http', '5000'])
    time.sleep(4)
    response = requests.get("http://localhost:4040/api/tunnels")
    tunnel_info = response.json()
    public_url = tunnel_info["tunnels"][0]["public_url"]
    print(" * ngrok tunnel URL:", public_url)
    return public_url

@app.route('/')
def index():
  session['search_result'] = [False,"",float(0)]
  return render_template('index.html')

@app.route('/image')
def image_page():
    return render_template('failed.html',reason = "you did not take a photo!!")
@app.route('/image', methods=['POST'])
def image():
  current_time = time.time()
  filename = str(current_time) + ".jpeg"
  imagedata = request.data[23:]
  with open(filename, "wb") as img:
    img.write(base64.decodebytes(imagedata))
  ##print(['search_result'])
  try:
    session['search'] = DeepFace.find(img_path= filename, db_path= filepath + "students", model_name= "ArcFace",  detector_backend = "retinaface", enforce_detection= False, anti_spoofing = True, refresh_database= False)[0]["identity"][0].split("/")[7]
    ##print(session['search'],session['search_result'])
    print(session.get("search","Guest"))
    search_result = [True,getStudentName(mysql,session.get("search","Guest")),current_time]
    session['search_result'] = search_result
  except Exception as error:
     search_result  = [False,"",current_time]
     if str(error) == "0":
        search_result[1] =  "you where not detected in the photo."
     elif str(error) == "Spoof detected in the given image.":
        search_result[1] =  "you tried to spoof the algorithim."
     else:
      print("An error occurred:", type(error).__name__, "–", error)
  session['search_result'] = search_result
  print(search_result)
  return render_template("failed.html",reason = "you where not detected in the photo.")

@app.route('/attended')
def attended():
  search_result = session.get("search_result","Guest")
  print(search_result)
  if str(search_result[1]) == "['']":
    return render_template('failed.html',reason = "you did not take a photo!!")
  #session.pop('search_result',None)
  if search_result[0] == True:
     return render_template("thanks.html",name = search_result[1],event = "insert event here" ,time = time.strftime('%H:%M', time.gmtime(search_result[2])))
  else:
     return render_template("failed.html",reason = search_result[1])
  ##return render_template("failed.html",reason = search_result[1])

@app.route('/is_done')
def is_done():
  search_result = session.get("search_result","Guest")
  if search_result[1] == "":
    return "", 202
  else:
    print(search_result[1]+"200"+str(search_result))
    return "", 200

@app.route('/login')
def login_page():
    return render_template('login.html')
@app.route('/login', methods=['POST'])
def login():
    username = request.form['username']
    password = request.form['password']
    if validateTeacher(db = mysql,
                                    bcrypt = bcrypt,
                                    teacherName = username,
                                    password=password):
        session["teacher_id"] = getID(username,10000000)
        return redirect(url_for('dashboard'))
    else:
        return "<script> alert('Incorrect username or password!');window.location.replace('/login');</script>"


@app.route('/register',methods = ['POST'])
def register():
    username = request.form['username']
    password = request.form['password']
    with app.app_context():
        if AddTeacher(db= mysql,teacherName=username,password=password):
            print("added teacher successfully")
        else:
            print("failed to add teacher %s",username)

@app.route('/dashboard',methods = ['GET','POST'])
def dashboard():
    if 'teacher_id' not in session:
        return "<script> alert('You are not in session'); window.location.replace('/login');</script>"

    cur = mysql.connection.cursor()
    cur.execute("SELECT * from student;")
    students = cur.fetchall()
    print(students)
    cur.close()

    if request.method == 'POST':
      selected_student = request.form.get('student')
      getAttendancePie(mysql,selected_student)
      return render_template('attendance.html')
    return render_template('dashboard.html', students=students)

@app.route('/logout', methods=['GET'])
def logout():
    session.pop('teacher_id', None)
    return "<script> alert('You have been logged out'); window.location.replace('/');</script>"

if __name__ == '__main__':
    public_url = start_ngrok()
    app.run(port=5000)

 * ngrok tunnel URL: https://a63f-34-83-116-119.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:52:21] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:52:22] "GET /static/styles.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:52:22] "GET /static/webcam.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:52:22] "GET /static/script.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:52:22] "GET /favicon.ico HTTP/1.1" 404 -


24-12-05 12:52:28 - Could be some changes in /content/drive/MyDrive/Colab/attendance/students not tracked.Set refresh_database to true to assure that any changes will be tracked.
24-12-05 12:52:29 - Searching 1733403148.9223785.jpeg in 3 length datastore


INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:52:29] "GET /is_done HTTP/1.1" 202 -
INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:52:30] "GET /is_done HTTP/1.1" 202 -
INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:52:31] "GET /is_done HTTP/1.1" 202 -
INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:52:31] "GET /is_done HTTP/1.1" 202 -
INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:52:32] "GET /is_done HTTP/1.1" 202 -
INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:52:33] "GET /is_done HTTP/1.1" 202 -
INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:52:34] "GET /is_done HTTP/1.1" 202 -
INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:52:34] "GET /is_done HTTP/1.1" 202 -
INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:52:35] "GET /is_done HTTP/1.1" 202 -
INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:52:36] "GET /is_done HTTP/1.1" 202 -
INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:52:37] "GET /is_done HTTP/1.1" 202 -
INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:52:37] "GET /is_done HTTP/1.1" 202 -
INFO:werkzeug:127.0.0.1 - - 

24-12-05 12:52:38 - find function duration 9.948365688323975 seconds
343490


...

INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:52:45] "GET /is_done HTTP/1.1" 200 -


Vivek200[True, 'Vivek', 1733403148.9223785]


INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:52:45] "GET /attended HTTP/1.1" 200 -


[True, 'Vivek', 1733403148.9223785]


INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:52:45] "GET /static/styles.css HTTP/1.1" 304 -


In [ ]:
filepath = userdata.get("attendance_filepath")
ngrok.set_auth_token(userdata.get("ngrok_token"))
app = Flask(__name__,template_folder= filepath+"templates",static_folder=filepath+"static")
bcrypt = Bcrypt(app)
mysql = DBconnect(app)


@app.route('/')
def index():
   cur = mysql.connection.cursor()
   cur.execute("SELECT * from student;")
   students = cur.fetchall()
   for student in students:
      print(student)
   student = (students[0][0])
   s = getStudentName(mysql,student)
   cur.close()
   print("Student ID : "+ str(student) +" Student name : " + s)

   return render_template('dashboard.html')

if __name__ == '__main__':
    public_url = start_ngrok()
    app.run(port=5000)


 * ngrok tunnel URL: https://83ae-34-83-116-119.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:__main__:Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "<ipython-input-68-75f30d59e000>", line 18, in index
    print("Student ID : "+ str(student) +" Student name : " + s)
T

(343490, 70, 'vivek')
(3686944, 1, 'freddy')
(3770361, 46, 'thomas')
(6239788, 8, 'lawrence')
(7754139, 1, 'michael')
(8633023, 1, 'jason')
(9968273, 110, 'altin')


INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 12:23:04] "GET /favicon.ico HTTP/1.1" 404 -
